In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
data=pd.read_csv('Resources/aviation_renamed.csv')
data.head()

,Unnamed: 0,Date,Country,City_State,Make,Model,Injury_Severity,Aircraft_Damage,Latitude,Longitude,Make and Model
0,0,12/31/2019,United States,"Elk, CA",cessna,175,Non-Fatal,Substantial,39.128611,-123.715833,cessna 175
1,1,12/31/2019,United States,"OLATHE, KS",mooney,M20S,Fatal(2),Destroyed,38.846111,-94.736111,mooney m20s
2,2,12/31/2019,United States,"Fairbanks, AK",cessna,170,Non-Fatal,Substantial,64.666945,-148.133334,cessna 170
3,3,12/28/2019,United States,"LAFAYETTE, LA",piper,PA 31T,Fatal(5),Destroyed,30.176111,-92.007500,piper pa 31t
4,4,12/25/2019,United States,"Headland, AL",bell,407,Fatal(1),Substantial,31.364167,-85.312500,bell 407


In [4]:
planes=pd.DataFrame({'Make and Model':[],
                     'Make and Model Guess':[],
                     'Description':[],
                     'URL':[]
                     })
nolist=pd.DataFrame({'Plane':[]})
searchlist=pd.DataFrame({'Plane':[]})

In [5]:
Make=data['Make'].value_counts()
Make=pd.DataFrame(Make)
Make=Make.reset_index(drop=False).rename(columns={'index':'Make', 'Make': 'count'})
Make

,Make,count
0,cessna,7561
1,piper,4275
2,beechcraft,1635
3,robinson,760
4,bell,695
...,...,...
1175,powrachute corp.,1
1176,allied ag cat productions,1
1177,s.n.i.a.s,1
1178,mx aircraft llc,1


In [6]:
makemodel=data['Make and Model'].value_counts()
makemodel=pd.DataFrame(makemodel)
makemodel=makemodel.reset_index(drop=False).rename(columns={'index':'Make And Model', 'Make and Model': 'count'})
makemodel

,Make And Model,count
0,cessna 172,585
1,cessna 152,381
2,cessna 172s,324
3,cessna 172n,316
4,cessna 180,229
...,...,...
4798,siai-marchetti sf.260tp,1
4799,aerolite bearcat,1
4800,bell b206l-1,1
4801,dromader m-18,1


In [7]:
for i in range(len(makemodel)):
    a=0
    base_url='https://en.wikipedia.org/wiki/Special:Search?search='
    mm2=makemodel['Make And Model'][i]
    mm=makemodel['Make And Model'][i].replace(' ','+').replace('inc','')
    url2=mm+'&go=Go&ns0=1'
    url=base_url+url2
    browser.visit(url)
    html = browser.html
    searchlink=html
    soup = BeautifulSoup(html, 'html.parser')
    mm_df=pd.DataFrame({'Plane':[mm2]})

    try:
        articles = soup.find('p')
        if len(articles.text.strip())==0:
            articles = soup.find_all('p')[1]
        text=articles.text.strip().replace('\n',' ').replace('\xa0','')
        script=soup.body.find_all('script', type="application/ld+json")[0]
        wiki_link=json.loads(script.string)['url']
        guess=soup.body.find_all('h1', id="firstHeading")[0].text
        plane=pd.DataFrame({'Make and Model':[makemodel['Make And Model'][i]],
                            'Make and Model Guess':[guess],
                            'Description':[text],
                            'URL':[wiki_link]
                            })
        planes=planes.append(plane)
    except:
        searchlist=searchlist.append(mm_df)
        try:
            article2 = soup.find('div', class_='mw-search-result-heading')
            click=article2.a.text.replace(' ','_')
            browser.click_link_by_href('/wiki/'+click)
            time.sleep(1)
        except:
            print('No clickable link!')
            a=1
        try:
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            article2 = soup.find('p')
            if len(article2.text.strip())==0:
                article2 = soup.find_all('p')[1]
            text=article2.text.strip().replace('\n',' ').replace('\xa0','')
            script=soup.body.find_all('script', type="application/ld+json")[0]
            wiki_link=json.loads(script.string)['url']
            guess=soup.body.find_all('h1', id="firstHeading")[0].text
            plane=pd.DataFrame({'Make and Model':[makemodel['Make And Model'][i]],
                                'Make and Model Guess':[guess],
                                'Description':[text],
                                'URL':[wiki_link]
                                })
            planes=planes.append(plane)
        except:
            print(f'did not work:{mm2}')
            nolist=nolist.append(mm_df)

            
planes.head()

not work:cameron balloons us v-90
No clickable link!
did not work:s.n.i.a.s. as-350b ecureuil
No clickable link!
did not work:convair div. of gen. dynamics 340
No clickable link!
did not work:remos aircraft gmbh gemini remos
No clickable link!
did not work:sikorsky s-55t
No clickable link!
did not work:thrush aircraft inc s2rhg-t65
No clickable link!
did not work:jabiru usa sport aircraft, llc j250-sp
No clickable link!
did not work:columbia aircraft mfg. lc41-550fg
No clickable link!
did not work:mccleery rans s6es
No clickable link!
did not work:vans rv-12
No clickable link!
did not work:rans rv 12
No clickable link!
did not work:arbc inc dba lindstrand 180a
No clickable link!
did not work:ferguson william c fx4-2
No clickable link!
did not work:aero vodochody aero. works l-29 delfin
No clickable link!
did not work:north wing uum inc sport x2
No clickable link!
did not work:eclipse aviation hxb
No clickable link!
did not work:smithwick/treidel tukan
No clickable link!
did not work:ev

,Make and Model,Make and Model Guess,Description,URL
0,cessna 172,Cessna 172,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
0,cessna 152,Cessna 152,"The Cessna 152 is an American two-seat, fixed-...",https://en.wikipedia.org/wiki/Cessna_152
0,cessna 172s,Cessna 172,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
0,cessna 172n,Cessna 172,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
0,cessna 180,Cessna 180,"The Cessna 180 is a four- or six-seat, fixed c...",https://en.wikipedia.org/wiki/Cessna_180


In [13]:
planes=planes.reset_index(drop=True)

In [24]:
planes.to_csv('Resources/aircraft_list_raw2.csv')

,Make and Model,Make and Model Guess,Description,URL
0,cessna 172,Cessna 172,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
1,cessna 152,Cessna 152,"The Cessna 152 is an American two-seat, fixed-...",https://en.wikipedia.org/wiki/Cessna_152
2,cessna 172s,Cessna 172,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
3,cessna 172n,Cessna 172,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
4,cessna 180,Cessna 180,"The Cessna 180 is a four- or six-seat, fixed c...",https://en.wikipedia.org/wiki/Cessna_180


In [20]:
planes_edit=planes.drop('Make and Model Guess', axis = 1)
planes_edit.to_csv('Resources/aircraft_list.csv')

In [31]:
planes_edit2=pd.DataFrame({'Make and Model':nolist['Plane'],
                           'Description':['']*len(nolist['Plane']),
                           'URL':['']*len(nolist['Plane'])})
planes_edit2=planes_edit.append(planes_edit2)
planes_edit2=planes_edit2.reset_index(drop=True)
planes_edit2.to_csv('Resources/aircraft_list.csv')

In [9]:
#Run these to find trouble cases

#nolist.to_csv('Resources/No_list.csv')
#searchlist.to_csv('Resources/Search_List.csv')